In [1]:
import pandas as pd
import numpy as np
import sys,time
from numba import jit
from math import sqrt
import math

floor = math.floor

In [2]:
# Loading DataSet
df_test = pd.read_csv('data/Result_Gamma/Gamma_data_All.csv')
df_test.rename(columns = {'Unnamed: 0':'index'},inplace =True)
df_test.set_index('index')
df_test.iloc[:8]

,index,x,y,z,Edep,PhotonNums,GridID,SIPM_Signal
0,0,-7.86084,1.844020,7.28039,0.511,22909.0,6,[0.01063307 0.01051034 0.0107467 0.00870494 0...
1,1,2.53285,0.615561,2.95402,0.511,25934.0,7,[0.010286 0.00986266 0.0112363 0.01110092 0...
2,2,3.49118,-7.480860,3.28973,0.511,26801.0,5,[0.0109348 0.00968922 0.01082942 0.00991025 0...
3,3,3.04109,-7.029700,2.06908,0.511,21247.0,5,[0.00925904 0.01091226 0.01043416 0.01092588 0...
4,4,-4.08897,-3.216470,8.62749,0.511,29193.0,4,[0.01208897 0.01103855 0.01076774 0.00968498 0...
5,5,-7.74779,2.970800,-5.89749,0.511,29164.0,2,[0.01623609 0.01740508 0.0171002 0.01269068 0...
6,6,-8.32235,3.183920,-2.58078,0.511,24369.0,2,[0.01640599 0.01610144 0.01425918 0.01250189 0...
7,7,6.61927,1.831820,-3.75156,0.511,23214.0,3,[0.00967651 0.01405791 0.01212597 0.01521115 0...


In [44]:
DataSet_A = np.load('data/Result_Gamma/Gamma_data_DataSetA_500000.npy')
DataSet_A.shape

(500000, 72)

In [3]:
# Calculate index_xyz for the first step
def Cal_Real_Grid_ID(row):
    L = 20 #mm
    N_x = 2
    N_y = 2
    N_z = 2
    index_x = floor((N_x * row['x'] / L) + N_x / 2)  # i = x * N/L + N/2, N_x = PositionNumsXYZ[0], L = 20mm
    index_y = floor((N_y * row['y'] / L) + N_y / 2)  #N_y = PositionNumsXYZ[1]
    index_z = floor((N_z * row['z'] / L) + N_z / 2)  #N_z = PositionNumsXYZ[2]
#     GridID = index_x + index_y * N_y + index_z * N_z * N_y
    return index_x,index_y,index_z

# Executing 
print("==================================================================")
print("---------------------------------4--------------------------------")
print("==================================================================")
print('Check x,y,z != 10, if ==10:',df_test[df_test['x'] ==10].shape,df_test[df_test['y'] ==10].shape,df_test[df_test['z'] ==10].shape)
print('Griding by Method 2...............')
t1 = time.time()
df_test['index_xyz'] = df_test.apply(Cal_Real_Grid_ID,axis = 1)   ### 1.3s/10000events
t2 = time.time()

print('Time Exhausted:',t2 - t1,'s')


df_test.iloc[:5]  

---------------------------------4--------------------------------
Check x,y,z != 10, if ==10: (0, 8) (0, 8) (0, 8)
Griding by Method 2...............
Time Exhausted: 48.728636741638184 s


,index,x,y,z,Edep,PhotonNums,GridID,SIPM_Signal,index_xyz
0,0,-7.86084,1.844020,7.28039,0.511,22909.0,6,[0.01063307 0.01051034 0.0107467 0.00870494 0...,"(0, 1, 1)"
1,1,2.53285,0.615561,2.95402,0.511,25934.0,7,[0.010286 0.00986266 0.0112363 0.01110092 0...,"(1, 1, 1)"
2,2,3.49118,-7.480860,3.28973,0.511,26801.0,5,[0.0109348 0.00968922 0.01082942 0.00991025 0...,"(1, 0, 1)"
3,3,3.04109,-7.029700,2.06908,0.511,21247.0,5,[0.00925904 0.01091226 0.01043416 0.01092588 0...,"(1, 0, 1)"
4,4,-4.08897,-3.216470,8.62749,0.511,29193.0,4,[0.01208897 0.01103855 0.01076774 0.00968498 0...,"(0, 0, 1)"


In [58]:
df_test['index_xyz'][10][0]

0

In [4]:
# Griding for the second step
def Cal_Real_Grid_ID_After(row):
    L = 20 #mm
    N_x = 2
    N_y = 2
    N_z = 2
    # Step Two
    L_x_after = L/N_x
    L_y_after = L/N_y
    L_z_after = L/N_z
    N_x_after = 4
    N_y_after = 4
    N_z_after = 4
    index_x = floor((row['x'] - row['index_xyz'][0] * L/N_x + L/2) * N_x_after/L_x_after)  # i = (x - index_x*L/N + L/2)* N_x_after/L_x_after
    index_y = floor((row['y'] - row['index_xyz'][1] * L/N_y + L/2) * N_y_after/L_y_after)
    index_z = floor((row['z'] - row['index_xyz'][2] * L/N_z + L/2) * N_z_after/L_z_after)
    GridID_after = index_x + index_y * N_y_after + index_z * N_z_after * N_y_after
    return GridID_after
#     return index_x,index_y,index_z,GridID_after

# Executing 
print("==================================================================")
print("---------------------------------4--------------------------------")
print("==================================================================")
print('Check x,y,z != 10, if ==10:',df_test[df_test['x'] ==10].shape,df_test[df_test['y'] ==10].shape,df_test[df_test['z'] ==10].shape)
print('Griding by Method 2...............')
t1 = time.time()
df_test['GridID_after'] = df_test.apply(Cal_Real_Grid_ID_After,axis = 1)   ### 99s/50,0000events
t2 = time.time()
print('Time Exhausted:',t2 - t1,'s')

df_test.iloc[:10]  

---------------------------------4--------------------------------
Check x,y,z != 10, if ==10: (0, 9) (0, 9) (0, 9)
Griding by Method 2...............
Time Exhausted: 99.8903534412384 s


,index,x,y,z,Edep,PhotonNums,GridID,SIPM_Signal,index_xyz,GridID_after
0,0,-7.86084,1.844020,7.28039,0.511,22909.0,6,[0.01063307 0.01051034 0.0107467 0.00870494 0...,"(0, 1, 1)",32
1,1,2.53285,0.615561,2.95402,0.511,25934.0,7,[0.010286 0.00986266 0.0112363 0.01110092 0...,"(1, 1, 1)",17
2,2,3.49118,-7.480860,3.28973,0.511,26801.0,5,[0.0109348 0.00968922 0.01082942 0.00991025 0...,"(1, 0, 1)",21
3,3,3.04109,-7.029700,2.06908,0.511,21247.0,5,[0.00925904 0.01091226 0.01043416 0.01092588 0...,"(1, 0, 1)",5
4,4,-4.08897,-3.216470,8.62749,0.511,29193.0,4,[0.01208897 0.01103855 0.01076774 0.00968498 0...,"(0, 0, 1)",58
5,5,-7.74779,2.970800,-5.89749,0.511,29164.0,2,[0.01623609 0.01740508 0.0171002 0.01269068 0...,"(0, 1, 0)",20
6,6,-8.32235,3.183920,-2.58078,0.511,24369.0,2,[0.01640599 0.01610144 0.01425918 0.01250189 0...,"(0, 1, 0)",36
7,7,6.61927,1.831820,-3.75156,0.511,23214.0,3,[0.00967651 0.01405791 0.01212597 0.01521115 0...,"(1, 1, 0)",34
8,8,-9.67000,8.261470,9.41787,0.511,21265.0,6,[0.01125195 0.01117209 0.01061858 0.01021157 0...,"(0, 1, 1)",60
9,9,-5.18280,-3.811480,-6.29035,0.511,23977.0,0,[0.03519093 0.03698767 0.02674402 0.01648585 0...,"(0, 0, 0)",25


In [59]:
for s in np.arange(DataSet_A.shape[1]):
    df_test['Channel{:.0f}'.format(s)] = DataSet_A[:,s]
    
# df_test.drop('SIPM_Signal',axis = 1,inplace = True)
df_test.iloc[:10]

,index,x,y,z,Edep,PhotonNums,GridID,index_xyz,GridID_after,Channel0,...,Channel62,Channel63,Channel64,Channel65,Channel66,Channel67,Channel68,Channel69,Channel70,Channel71
0,0,-7.86084,1.844020,7.28039,0.511,22909.0,6,"(0, 1, 1)",32,0.010633,...,0.019548,0.012087,0.010709,0.012575,0.031921,0.031334,0.020093,0.012107,0.009313,0.010461
1,1,2.53285,0.615561,2.95402,0.511,25934.0,7,"(1, 1, 1)",17,0.010286,...,0.016812,0.022520,0.021491,0.017042,0.010847,0.013315,0.015357,0.017552,0.019183,0.013775
2,2,3.49118,-7.480860,3.28973,0.511,26801.0,5,"(1, 0, 1)",21,0.010935,...,0.012804,0.013468,0.013229,0.011198,0.011884,0.011826,0.010802,0.012025,0.011555,0.010947
3,3,3.04109,-7.029700,2.06908,0.511,21247.0,5,"(1, 0, 1)",5,0.009259,...,0.012700,0.014163,0.013421,0.010870,0.010662,0.010815,0.012064,0.013032,0.010909,0.010216
4,4,-4.08897,-3.216470,8.62749,0.511,29193.0,4,"(0, 0, 1)",58,0.012089,...,0.015701,0.014621,0.012878,0.011292,0.014258,0.017751,0.016231,0.012946,0.013719,0.011093
5,5,-7.74779,2.970800,-5.89749,0.511,29164.0,2,"(0, 1, 0)",20,0.016236,...,0.009288,0.009603,0.009537,0.010763,0.012205,0.010945,0.010337,0.009850,0.010892,0.010500
6,6,-8.32235,3.183920,-2.58078,0.511,24369.0,2,"(0, 1, 0)",36,0.016406,...,0.010309,0.008517,0.008725,0.010019,0.012620,0.011309,0.009546,0.010193,0.010448,0.009868
7,7,6.61927,1.831820,-3.75156,0.511,23214.0,3,"(1, 1, 0)",34,0.009677,...,0.008805,0.009687,0.012501,0.012545,0.010424,0.009384,0.010193,0.010055,0.011573,0.011277
8,8,-9.67000,8.261470,9.41787,0.511,21265.0,6,"(0, 1, 1)",60,0.011252,...,0.008567,0.007813,0.008082,0.012161,0.201323,0.016927,0.011281,0.009348,0.008192,0.012039
9,9,-5.18280,-3.811480,-6.29035,0.511,23977.0,0,"(0, 0, 0)",25,0.035191,...,0.011367,0.010737,0.011710,0.010185,0.011056,0.011950,0.009892,0.009415,0.010263,0.009093


In [78]:
df_ll = df_test[df_test['GridID'] == 1]
np.array(df_ll.iloc[0,9:])

array([0.011423303, 0.010346453, 0.00982391, 0.012468179, 0.031546578,
       0.04056017, 0.009749279, 0.009120707, 0.009015301, 0.014298983,
       0.03952739, 0.044826556, 0.010846802, 0.009520369, 0.009190973,
       0.009293956, 0.025278937, 0.03250514, 0.010005187, 0.009997252,
       0.008820793, 0.008308494, 0.011355256, 0.015166001, 0.010381702,
       0.01025714, 0.008502102, 0.009048137, 0.008526061, 0.009535321,
       0.010916264, 0.011759, 0.010423535, 0.011024958, 0.011184141,
       0.011925094, 0.009768062, 0.012626925, 0.015304467, 0.0148128215,
       0.01566146, 0.015359565, 0.0104593355, 0.013499399, 0.016424518,
       0.0167076, 0.01668084, 0.016356133, 0.010221343, 0.012897352,
       0.014218798, 0.014529385, 0.016283184, 0.015526251, 0.00989856,
       0.011364585, 0.013616035, 0.01478852, 0.015388751, 0.014368393,
       0.009245416, 0.010483565, 0.011895777, 0.013945672, 0.014227548,
       0.012406999, 0.009664885, 0.010808892, 0.009704061, 0.011874867,
    

In [81]:
df_name = []
DataSet_name = []
for i in np.arange(8):
    df_name.append('df_test_after_cell{:.0f}'.format(i))
    DataSet_name.append('DataSet_A_after_cell{:.0f}'.format(i))
    print(df_name[i],DataSet_name[i])
    df_name[i] = df_test[df_test['GridID'] == i]
    df_name[i] = df_name[i].reset_index(drop=True)  #Reset index
    print(df_name[i].shape)

    DataSet_name[i] = np.zeros(shape = (df_name[i].shape[0],6*6*2),dtype = float)
    for n in range(df_name[i].shape[0]):
        DataSet_name[i][n,:] = np.array(df_name[i].iloc[n,9:])

    print(df_name[i].shape)

df_test_after_cell0 DataSet_A_after_cell0
(46918, 81)
(46918, 81)
df_test_after_cell1 DataSet_A_after_cell1
(47199, 81)
(47199, 81)
df_test_after_cell2 DataSet_A_after_cell2
(47128, 81)
(47128, 81)
df_test_after_cell3 DataSet_A_after_cell3
(47309, 81)
(47309, 81)
df_test_after_cell4 DataSet_A_after_cell4
(77737, 81)
(77737, 81)
df_test_after_cell5 DataSet_A_after_cell5
(78162, 81)
(78162, 81)
df_test_after_cell6 DataSet_A_after_cell6
(77794, 81)
(77794, 81)
df_test_after_cell7 DataSet_A_after_cell7
(77753, 81)
(77753, 81)


In [22]:
def Cal_Dataset_after(df_tem):
    print(df_tem.shape[0])
    DataSet_tem = np.zeros(shape = (df_tem.shape[0],6*6*2),dtype = float)
    for n in range(df_tem.shape[0]):
#         DataSet_tem[n,:] = df_tem['SIPM_Signal'][n]
        print(df_tem['SIPM_Signal'][n])
    return DataSet_tem

DataSet_Cell0 = Cal_Dataset_after(df_name[0])
    

46918


KeyError: 0

In [ ]:
print('Counting the real IPD by Method 1..............')
t5_1_1 = time.time()
df_grouped = df_test.groupby('GridID',as_index = False).count() #count
GridID_index = np.array(df_grouped['GridID'])
GridID_Distri_P = np.array(df_grouped['x'] / Gamma_Num)
Real_Distri_P = np.zeros(shape = (Position_Nums),dtype = float)
Real_Distri_P[GridID_index] = GridID_Distri_P # p_i
t5_1_2 = time.time()

In [43]:
df_ll = df_test[df_test['GridID'] == 1]
df_ll = df_ll.reset_index(drop=True)  #Reset index
np.array(df_ll['SIPM_Signal'][0])

array('[0.0114233  0.01034645 0.00982391 0.01246818 0.03154658 0.04056017\n 0.00974928 0.00912071 0.0090153  0.01429898 0.03952739 0.04482656\n 0.0108468  0.00952037 0.00919097 0.00929396 0.02527894 0.03250514\n 0.01000519 0.00999725 0.00882079 0.00830849 0.01135526 0.015166\n 0.0103817  0.01025714 0.0085021  0.00904814 0.00852606 0.00953532\n 0.01091626 0.011759   0.01042353 0.01102496 0.01118414 0.01192509\n 0.00976806 0.01262692 0.01530447 0.01481282 0.01566146 0.01535956\n 0.01045934 0.0134994  0.01642452 0.0167076  0.01668084 0.01635613\n 0.01022134 0.01289735 0.0142188  0.01452938 0.01628318 0.01552625\n 0.00989856 0.01136459 0.01361603 0.01478852 0.01538875 0.01436839\n 0.00924542 0.01048357 0.01189578 0.01394567 0.01422755 0.012407\n 0.00966488 0.01080889 0.00970406 0.01187487 0.01219941 0.0103012 ]',
      dtype='<U800')